# BipedalWalker-v3 Reinforcement Learning Training

Обучение агента BipedalWalker с использованием алгоритмов SAC, TD3, PPO

**Перед запуском:**
1. Runtime -> Change runtime type -> T4 GPU
2. Запускайте ячейки последовательно

## 1. Проверка GPU

In [ ]:
!nvidia-smi

## 2. Установка зависимостей

In [ ]:
# Установка системных зависимостей для Box2D
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y -qq swig build-essential python3-dev > /dev/null 2>&1

# Установка Box2D отдельно
!pip install -q box2d-py

# Установка остальных пакетов
!pip install -q stable-baselines3 gymnasium pygame tensorboard imageio imageio-ffmpeg

print("✓ Все пакеты установлены!")

## 3. Импорты и настройка

In [ ]:
import gymnasium as gym
import numpy as np
import torch
from stable_baselines3 import SAC, TD3, PPO
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
import matplotlib.pyplot as plt
import imageio
import os
from datetime import datetime
from IPython.display import Video, HTML, display
import warnings
warnings.filterwarnings('ignore')

# Проверка GPU
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU не обнаружена! Используется CPU.")

# Создание директорий
!mkdir -p models logs tensorboard_logs results videos
print("\n✓ Директории созданы")

## 4. Тестирование окружения

In [ ]:
# Тест окружения
env = gym.make('BipedalWalker-v3')
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

# Тестовый эпизод
obs, info = env.reset()
for _ in range(5):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    if done or truncated:
        break
env.close()
print("\n✓ Окружение работает!")

## 5. Функции для тренировки

In [ ]:
def make_env():
    """Создание окружения"""
    env = gym.make('BipedalWalker-v3')
    env = Monitor(env)
    return env

def train_agent(algorithm='SAC', timesteps=500000, save_freq=50000):
    """
    Тренировка агента
    
    Args:
        algorithm: 'SAC', 'TD3', или 'PPO'
        timesteps: Количество шагов обучения
        save_freq: Частота сохранения чекпоинтов
    """
    print(f"\n{'='*60}")
    print(f"Тренировка {algorithm}")
    print(f"Шагов: {timesteps:,}")
    print(f"{'='*60}\n")
    
    # Создание окружений
    env = make_env()
    eval_env = make_env()
    
    # Пути для сохранения
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_dir = f"models/{algorithm.lower()}_{timestamp}"
    log_dir = f"tensorboard_logs/{algorithm.lower()}_{timestamp}"
    
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    
    # Коллбэки
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=f"{model_dir}/best",
        log_path=f"logs/{algorithm.lower()}",
        eval_freq=10000,
        deterministic=True,
        render=False,
        n_eval_episodes=5
    )
    
    checkpoint_callback = CheckpointCallback(
        save_freq=save_freq,
        save_path=f"{model_dir}/checkpoints",
        name_prefix=f"{algorithm.lower()}_model"
    )
    
    # Создание модели
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Используется устройство: {device}\n")
    
    if algorithm == 'SAC':
        model = SAC(
            'MlpPolicy', env, verbose=1,
            learning_rate=3e-4, buffer_size=300000,
            batch_size=256, gamma=0.99, tau=0.02,
            tensorboard_log=log_dir, device=device
        )
    elif algorithm == 'TD3':
        model = TD3(
            'MlpPolicy', env, verbose=1,
            learning_rate=1e-3, buffer_size=200000,
            batch_size=100, gamma=0.99, tau=0.005,
            tensorboard_log=log_dir, device=device
        )
    elif algorithm == 'PPO':
        model = PPO(
            'MlpPolicy', env, verbose=1,
            learning_rate=3e-4, n_steps=2048,
            batch_size=64, n_epochs=10,
            gamma=0.99, gae_lambda=0.95, clip_range=0.2,
            tensorboard_log=log_dir, device=device
        )
    
    # Обучение
    print(f"Начало обучения...\n")
    model.learn(
        total_timesteps=timesteps,
        callback=[eval_callback, checkpoint_callback],
        progress_bar=True
    )
    
    # Сохранение
    final_model_path = f"{model_dir}/final_model"
    model.save(final_model_path)
    print(f"\n✓ Модель сохранена: {final_model_path}\n")
    
    env.close()
    eval_env.close()
    
    return model, model_dir

print("✓ Функции загружены")

## 6. Настройка параметров

**Выберите количество шагов:**
- 50000 - быстрый тест (~10-15 минут на алгоритм)
- 500000 - полное обучение (~1-1.5 часа на алгоритм)

In [ ]:
# НАСТРОЙТЕ ЭТО ЗНАЧЕНИЕ:
TIMESTEPS = 50000  # Быстрый тест
# TIMESTEPS = 500000  # Полное обучение (раскомментируйте для полного обучения)

print(f"Будет обучено: {TIMESTEPS:,} шагов на алгоритм")
print(f"Примерное время на алгоритм: {TIMESTEPS//5000} минут")
print(f"Всего для 3 алгоритмов: ~{TIMESTEPS//5000 * 3} минут")

## 7. Тренировка алгоритмов

### 7.1. SAC (Soft Actor-Critic)

In [ ]:
model_sac, dir_sac = train_agent('SAC', timesteps=TIMESTEPS)

### 7.2. TD3 (Twin Delayed DDPG)

In [ ]:
model_td3, dir_td3 = train_agent('TD3', timesteps=TIMESTEPS)

### 7.3. PPO (Proximal Policy Optimization)

In [ ]:
model_ppo, dir_ppo = train_agent('PPO', timesteps=TIMESTEPS)

## 8. Функция для оценки

In [ ]:
def evaluate_model(model, algorithm_name, n_episodes=10, render_video=True):
    """
    Оценка модели
    """
    print(f"\nОценка {algorithm_name}...")
    env = gym.make('BipedalWalker-v3', render_mode='rgb_array')
    
    episode_rewards = []
    frames = []
    
    for episode in range(n_episodes):
        obs, info = env.reset()
        done = False
        truncated = False
        episode_reward = 0
        
        while not (done or truncated):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            
            if render_video and episode == 0:
                frame = env.render()
                if frame is not None:
                    frames.append(frame)
        
        episode_rewards.append(episode_reward)
        print(f"  Эпизод {episode + 1}: {episode_reward:.2f}")
    
    env.close()
    
    # Сохранение видео
    video_path = None
    if frames:
        video_path = f'videos/{algorithm_name.lower()}_evaluation.mp4'
        imageio.mimsave(video_path, frames, fps=30)
        print(f"  ✓ Видео: {video_path}")
    
    # Статистика
    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)
    
    print(f"\n  Средняя награда: {mean_reward:.2f} ± {std_reward:.2f}")
    print(f"  Диапазон: [{np.min(episode_rewards):.2f}, {np.max(episode_rewards):.2f}]\n")
    
    return episode_rewards, video_path

print("✓ Функция оценки загружена")

## 9. Оценка моделей

In [ ]:
# Оценка всех моделей
print("="*60)
print("ОЦЕНКА МОДЕЛЕЙ")
print("="*60)

rewards_sac, video_sac = evaluate_model(model_sac, 'SAC', n_episodes=10)
rewards_td3, video_td3 = evaluate_model(model_td3, 'TD3', n_episodes=10)
rewards_ppo, video_ppo = evaluate_model(model_ppo, 'PPO', n_episodes=10)

print("✓ Оценка завершена")

## 10. Просмотр видео

In [ ]:
# Показать видео SAC
if video_sac:
    print("SAC:")
    display(Video(video_sac, embed=True, width=600))

In [ ]:
# Показать видео TD3
if video_td3:
    print("TD3:")
    display(Video(video_td3, embed=True, width=600))

In [ ]:
# Показать видео PPO
if video_ppo:
    print("PPO:")
    display(Video(video_ppo, embed=True, width=600))

## 11. Сравнение алгоритмов

In [ ]:
# График сравнения
algorithms = ['SAC', 'TD3', 'PPO']
mean_rewards = [
    np.mean(rewards_sac),
    np.mean(rewards_td3),
    np.mean(rewards_ppo)
]
std_rewards = [
    np.std(rewards_sac),
    np.std(rewards_td3),
    np.std(rewards_ppo)
]

plt.figure(figsize=(10, 6))
x_pos = np.arange(len(algorithms))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

bars = plt.bar(x_pos, mean_rewards, yerr=std_rewards,
               align='center', alpha=0.8, ecolor='black',
               capsize=10, color=colors)

plt.xlabel('Алгоритм', fontsize=12, fontweight='bold')
plt.ylabel('Средняя награда', fontsize=12, fontweight='bold')
plt.title('Сравнение производительности алгоритмов RL\nBipedalWalker-v3', 
         fontsize=14, fontweight='bold')
plt.xticks(x_pos, algorithms, fontsize=11)
plt.grid(True, alpha=0.3, axis='y')

# Значения над столбцами
for bar, mean, std in zip(bars, mean_rewards, std_rewards):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{mean:.1f}±{std:.1f}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('results/comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ График сохранен: results/comparison.png")

## 12. Итоговая таблица

In [ ]:
import pandas as pd

# Создание итоговой таблицы
results_df = pd.DataFrame({
    'Алгоритм': ['SAC', 'TD3', 'PPO'],
    'Средняя награда': [np.mean(rewards_sac), np.mean(rewards_td3), np.mean(rewards_ppo)],
    'Std': [np.std(rewards_sac), np.std(rewards_td3), np.std(rewards_ppo)],
    'Мин': [np.min(rewards_sac), np.min(rewards_td3), np.min(rewards_ppo)],
    'Макс': [np.max(rewards_sac), np.max(rewards_td3), np.max(rewards_ppo)]
})

results_df = results_df.round(2)
results_df = results_df.sort_values('Средняя награда', ascending=False)

print("\n" + "="*60)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("="*60)
print(results_df.to_string(index=False))
print("="*60)

# Сохранение
results_df.to_csv('results/results.csv', index=False)
print("\n✓ Результаты сохранены: results/results.csv")

## 13. TensorBoard

In [ ]:
# Загрузка TensorBoard в Colab
%load_ext tensorboard
%tensorboard --logdir tensorboard_logs

## 14. Скачивание результатов

In [ ]:
# Архивирование результатов
!zip -r -q results.zip models/ videos/ results/ logs/

print("✓ Результаты упакованы в results.zip")
print("\nСкачивание файла...")

# Скачивание
from google.colab import files
files.download('results.zip')

print("\n" + "="*60)
print("РАБОТА ЗАВЕРШЕНА!")
print("="*60)
print("✓ Обучены 3 алгоритма: SAC, TD3, PPO")
print("✓ Созданы видео работы агентов")
print("✓ Построены графики сравнения")
print("✓ Результаты сохранены в results.zip")
print("="*60)